# Static Webscraping with BeautifulSoup


In this notebook, I will walk you through the basics of webscraping. As we will see, webscraping, or programmitically extracting data (usually text) from a webpage, is a very common and useful part of a wide variety of natural language processing tasks. Often times, webscraping will be the first step in your NLP pipeline and so the results we get from webscraping have the potential to affect all of the subsequent steps of your pipeline. Too, smart webscraping can save you a lot of time upfront that you would have had to spend on optimization later on in the process.

## Our task in this notebook
I will take the Wikipedia page of the [list of all Roman Emperors](https://en.wikipedia.org/wiki/List_of_Roman_emperors) and recreate the tables in it in Python. This is a very common task in NLP. A lot of the time, we need to cross reference a source we are interested in with a secondary source like Wikipedia. I will show you the basics in this notebook.

## Goals
* Access a static webpage in Python using the requests library
* Navigate through raw html code to find the data we are interested in
* Arrange that data into a useful format
* Clean the data to fit whatever we want to do with it

In [ ]:
## we want to scrape all of the tables from this wikipedia article and make our own tables of the same same data
from IPython.display import IFrame

IFrame(
    "https://en.wikipedia.org/wiki/List_of_Roman_emperors#Principate_(27_BC_–_AD_284)",
    width=1000,
    height=500,
)

## `requests`
`requests` is an incredibly useful Python package that comes pre-installed on most distributions of Python. If you need to interact with the web from a .py file or notebook, you will be using requests in some form. Let's take a look at how it works.

In [ ]:
## note: although requests is very common, if this cell returns a ModuleNotFound error, uncomment the line below to install it
# !python3 -m pip install requests
import requests

In [ ]:
type(requests)

In [ ]:
## using the .get method we can access any webpage
r = requests.get("https://en.wikipedia.org/wiki/List_of_Roman_emperors")

## let's see what r is
type(r)

In [ ]:
## a Response object can give us all of the information we need
## we can call .text to see the raw html of a webpage
html = r.text
type(html)

## `BeautifulSoup`
Now that we have our HTML output, we need to parse it as a structured form of text. Above we see that our html object is a string, meaning we can only use the attributes and methods of strings on it. While we could create a parser from scratch that lets us take advantage of the structured nature of the HTML code, as with many tasks in Python, we don't need to because one already exists for us. There are many packages that parse HTML, but we'll be using the most popular, [Beautiful Soup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/).

*A note on copyright*
<br>
Just because you can access and scrape a website doesn't always mean you should. Most websites are public and they have to be to share the information they seek to, but that does not mean that it is free. Most data on the internet can be used without a problem. These sources are generally protected under certain licences like the MIT or Creative Commons Attribution-ShareAlike 3.0. Generally, if a site does not have one of these licenses, you should stay away. Stealing data this way can be dangerous and can break the terms of service of many sites.

In [ ]:
## bs4 is in most distributions of Python, but if this cell does not work try:
## !python3 -m pip install bs4
from bs4 import BeautifulSoup

In [ ]:
## the BeautifulSoup parser takes in any string and attempts to parse it as HTML (or XML)
soup = BeautifulSoup(r.text, features="html")
type(soup)

In [ ]:
## with this BeautifulSoup object, we can navigate through the tag soup in a systematic way
soup
## but now we need to isolate the data we want
## tranisition to Chrome Developer Tools/HTML highlighting

In [ ]:
len(soup.find_all("tbody"))

In [ ]:
## now we have a iterable of each tbody object
for tbody in soup.find_all("tbody"):
    print(tbody)
    print("-" * 20)

In [ ]:
## importantly each element of this iterable can be navigated like the whole tree did
tbody = soup.find_all("tbody")[0]
for tr in tbody.find_all("tr")[
    1:
]:  ## the first element (0th in the list) is the columns names, so I have gotten rid of it
    # print(tr)
    # print('_________')
    row = tr.find_all(["td", "th"])
    print(
        row[0].a["href"],
        row[1].b.get_text(),
        row[1].small.get_text(),
        row[2].get_text(),
        row[3].get_text(),
        row[4].get_text(),
        sep="\n",
    )
    print("_________")
## now we're getting somewhere

I am going to use a very versatile package called `re` or regular expressions (regex) to do some advanced string parsing. This regex function, finditer, takes in a pattern and a text and returns all of the times that pattern occurs in the text. These patterns can look very complicated, but, in this case, it is '(?<=\)', which means: *find all of the places between an end paraenthesis and the rest of the string*.
<br>

You can read more about regex [here](https://librarycarpentry.org/lc-data-intro-archives/04-regular-expressions/index.html) and you can play around with your own regex at [regex101](https://regex101.com/).  

In [ ]:
## going back to the full list of tbodys and populate a dictionary with all our data
import re

emp_dict = {}
for tbody in soup.find_all("tbody"):
    for tr in tbody.find_all("tr")[1:]:
        row = tr.find_all(["td", "th"])
        if len(row) == 5:  ## check if each row is of the correct length
            if not isinstance(row[0].a, type(None)):  ## check if each row has an image
                img_url = f"en.wikipedia.org{row[0].a['href']}"

                life_details = re.split(
                    "(?<=\))", row[4].get_text()
                )  ## here I am using regex to search for the place between an end parenthesis ')' and the rest of the string
                if (
                    len(life_details) > 1
                ):  ## checking if there is a parenthesis, if there isn't then the cause of death recorded
                    life_date = life_details[0]
                    cod = life_details[1]
                else:
                    life_date = life_details[0]
                    cod = "None found."

                if not isinstance(
                    row[1].small, type(None)
                ):  ## checking if there is a full name associated with an emperor
                    full_name = row[1].small.get_text()
                else:
                    full_name = "None found."

                emp_dict[row[1].b.get_text()] = [
                    img_url,
                    full_name,
                    row[2].get_text(),
                    row[3].get_text(),
                    life_date,
                    cod,
                ]

In [ ]:
emp_dict

In [ ]:
import pickle

with open("emp_dict.pickle", "wb") as handle:
    pickle.dump(emp_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

*Using `pandas`*
<br>
A dictionary is useful, but with more and more data, you'll find that a dictionary is to simple of a data structure for most webscraped data. A `pandas` dataframe is much more scalable and makes tabular data much easier to work with. That being said, one is NEVER supposed to fill a dataframe with a loop, as we filled the dictionary above. Instead, we can use the dictionary we created above and turn it into a dataframe. This way we can keep both forms of the data in case we need dictionary representation later.

In [ ]:
import pandas as pd  ## you will almost always see pandas imported like this, the 'pd' alias is a very useful shorthand

## Let's see what happens when we input the dictionary directly
pd.DataFrame(emp_dict)
## it's close but not quite what we wanted

In [ ]:
## pandas is programmed to look for numerical indices, which dictionaries (because they're an unordered data type) do not have
## we can coerce it though to accept string values as the index with the 'from_dict' method and the 'orient' keyword argument
## the 'reset_index' method will then turn our index into a column and give us an index for the rows
emp_df = pd.DataFrame.from_dict(emp_dict, orient="index").reset_index()
emp_df = emp_df.rename(
    columns={
        "index": "name",
        0: "img",
        1: "full_name",
        2: "reign",
        3: "succession",
        4: "life_dates",
        5: "cause_of_death",
    }
)  ## last, this is one way to rename columns
emp_df

Yay! 🎉 🥳 Our data has been scraped! 🥳 🎉
... but what can we do with it 🤔
<br>

Let's try to plot all of the ages of the emperors, as we have that data in the life_dates column

In [ ]:
## we have a slight problem though...
## Take Trajan's row for instance
string = emp_df.loc[emp_df["name"] == "Trajan"].life_dates.iloc[
    0
]  ## gets a contents of the life_dates cell in the Trajan row
print(string)
print(str.encode(string))
## what is \xc2\xa0??

It might not seem like it, but the difference between the two lines above will be very significant in cleaning our data to be used in visualizations.

These collections of letters and numbers preceded by a backslash are byte representations of characters at the index position of the characters themselves. In fact these representations are slightly different characters than they might seem and we will have to normalize them in order to interact with them. To put a long story short, these characters come from a different text encoding (ISO-8895-1) than what Python expects (utf-8), so we must convert these non-standard characters into their standard output. We can use a package called unidecode, which also comes with most distributions of Python.

In [ ]:
#!pip install unidecode
import unidecode

print(string)
print(unidecode.unidecode(string))
print(str.encode(unidecode.unidecode(string)))

In [ ]:
def getAges(life_dates):
    ld = unidecode.unidecode(life_dates)
    age = re.search(
        "(?<=aged )([0-9]+)|(?<=aged approx. )([0-9]+)", ld
    )  ## more regex to extract the age
    if age:
        return int(age.group(0))
    else:
        return None  ## there are some emperors for whom we have no dates for

In [ ]:
getAges(string)

In [ ]:
## apply takes in a function and applies it to all of the members of a column
emp_df["age"] = emp_df["life_dates"].apply(getAges)
emp_df["age"]

In [ ]:
import plotly.express as px

fig = px.scatter(x=emp_df["name"], y=emp_df["age"])
fig.show()

## Reviewing what we learned
* The basics of the `requests` library
* Navigating HTML using BeautifulSoup
* How to construct a dictionary for our data
* Turning that dictionary into a `pandas` dataframe
* Cleaning our data for a specific purpose with `.apply`

As a challenge, try to do what I did for the ages of the emperors, but with the length that they reigned for. This is a much more difficult question and can be done in a couple different ways. You will likely have to use the `datetime` package in Python. If you have trouble or just want to show off how you did it, feel free to reach out and let me know at peter.nadel@tufts.edu

# Thanks for reading